In [ ]:
from segmentation_client import SegmentationClient
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
imgPath = 'C:/Uni/1M. Semester/DocDig/Test/EntirePages'

In [ ]:
# List of images in the directory
image_files = os.listdir(imgPath)

# Create a figure with subplots
num_images = len(image_files)
fig, axes = plt.subplots(num_images, 1, figsize=(60,120))

for i, img in enumerate(image_files):
    # Load the image in Grayscale
    img_gray = cv.imread(os.path.join(imgPath, img), cv.IMREAD_GRAYSCALE)
    img_color = cv.imread(os.path.join(imgPath, img), cv.IMREAD_COLOR)
    assert img_gray is not None, "file could not be read, check with os.path.exists()"

    # Apply edge detection (e.g., using Canny)
    edges = cv.Canny(img_gray, 50, 150, apertureSize=3)
    #cv.imshow("Edges", edges)
    #cv.waitKey()
    #cv.destroyAllWindows()

    # Use Hough Line Transform to detect lines
    lines = cv.HoughLines(edges, 1, np.pi / 180, 700)

    # Create a copy of the original image to draw lines on
    img_lines = img_color.copy()
    #img_lines = np.zeros_like(img_gray)
    
    # Calculate angles and draw detected lines
    angles = []
    if lines is not None:
      for rho, theta in lines[:, 0]:
        # Convert angle from radians to degrees
        angle = np.rad2deg(theta)
        # Filter for nearly vertical lines (close to 0° or 180°)
        if angle < 10 or angle > 170:
          angles.append(angle if angle < 90 else angle - 180)

          # Draw the line on the image for visualization
          a = np.cos(theta)
          b = np.sin(theta)
          x0 = a * rho
          y0 = b * rho
          x1 = int(x0 + 1000 * (-b))
          y1 = int(y0 + 1000 * (a))
          x2 = int(x0 - 1000 * (-b))
          y2 = int(y0 - 1000 * (a))
          cv.line(img_lines, (x1, y1), (x2, y2), tuple(np.random.randint(0, 256, 3).tolist()), 2)

    # Count the number of detected lines
    num_lines = len(lines) if lines is not None else 0
    
    # Display each processed image in a subplot
    axes[i].imshow(cv.cvtColor(img_lines, cv.COLOR_BGR2RGB))
    axes[i].axis('off')  # Hide the axis
    axes[i].set_title(f"Image {i+1} - Lines: {num_lines}")

# Adjust layout and display the plot
plt.tight_layout()
plt.show()